In [2]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib

import SatCallbacks
import Clone_Model
import rsc

In [3]:
checkpoint_path = "./cloning/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [4]:
#      CLONE MODEL EAGER ENABLED
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 3

train, test = rsc.get_titanic_dataset()


model = rsc.get_model_slim()

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


history = model.fit(train,
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])


clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

sat_cb = Clone_Model.sat_results()

history = clone.fit(train,
                    batch_size=batch_size,
                    epochs = 1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback, sat_cb])
                

3/3 [==============================] - 0s 11ms/step - loss: 0.6874 - val_loss: 0.6890


TypeError: in user code:

    /home/leo/Desktop/Thesis/New_Repo/Clone_Model.py:101 call  *
        _o, _r, _s, = self.sat_layer(out)
    /home/leo/Desktop/Thesis/New_Repo/SatLayer.py:223 __call__  *
        self.build(input)
    /home/leo/Desktop/Thesis/New_Repo/SatLayer.py:129 build  *
        self.inits=[np.zeros(shape=(1,)),

    TypeError: 'NoneType' object cannot be interpreted as an integer


In [4]:
#      TRAIN MODEL EAGER ENABLED MNIST CLASSIFICATION
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 20

train, test = rsc.get_mnist()


model = rsc.get_model_unitlist(hidden_layers_spec=[128])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



history = model.fit(train[0],
                    train[1],
                    batch_size=batch_size,
                    epochs=5, 
                    steps_per_epoch=None, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])



Epoch 1/5
3000/3000 [==============================] - 8s 3ms/step - loss: 2.3708 - accuracy: 0.8456 - val_loss: 0.3684 - val_accuracy: 0.8500
Epoch 2/5
3000/3000 [==============================] - 10s 3ms/step - loss: 0.3940 - accuracy: 0.9053 - val_loss: 0.1320 - val_accuracy: 0.9500
Epoch 3/5
3000/3000 [==============================] - 8s 3ms/step - loss: 0.2923 - accuracy: 0.9279 - val_loss: 0.2223 - val_accuracy: 0.9000
Epoch 4/5
3000/3000 [==============================] - 9s 3ms/step - loss: 0.2652 - accuracy: 0.9341 - val_loss: 0.1348 - val_accuracy: 0.9500
Epoch 5/5
3000/3000 [==============================] - 11s 4ms/step - loss: 0.2502 - accuracy: 0.9400 - val_loss: 0.1377 - val_accuracy: 0.9500


In [10]:
#      CLONE MODEL EAGER ENABLED MNIST CLASSIFICATION
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)
importlib.reload(SatCallbacks)

print(tf.executing_eagerly())

clone = Clone_Model.satify_model(model)

"""clone.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              run_eagerly = tf.executing_eagerly())
"""
new_cb = SatCallbacks.sat_logger()

sat_cb = Clone_Model.sat_results()

history = clone.fit(train[0],
                    train[1],
                    batch_size=10,
                    epochs = 2,
                    steps_per_epoch=5,
                    validation_data=test, 
                    validation_steps=2, 
                    callbacks=[new_cb])
              

True
<class 'tensorflow.python.keras.layers.core.Flatten'>
input_shape: [None, 128]
layer_width: (128,)
input_shape: [None, 10]
layer_width: (10,)
Epoch 1/2
4/5 [=======================>......] - ETA: 0s - loss: 4.1785 - accuracy: 0.9000cross-replica-ctxt: False
tf.dist.strat:  <tensorflow.python.distribute.distribute_lib._DefaultDistributionStrategy object at 0x7f279025d790>

Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Tensors batch_k and batch_tf are equal! 
 


Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Tensors batch_k and batch_tf are equal! 
 


Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Tensors batch_k and batch_tf are equal! 
 


Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Tensors batch_k and batch_tf are equal! 
 


Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Tensors batch_k and 

Epoch 2/2
1/5 [=====>........................] - ETA: 0s - loss: 3.2382 - accuracy: 1.0000cross-replica-ctxt: False
tf.dist.strat:  <tensorflow.python.distribute.distribute_lib._DefaultDistributionStrategy object at 0x7f279025d790>

Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Mean Difference of elements: 0.6161899997387081
Avg Ratio Diff/Value of batch_k elements: nan
Avg Ratio Diff/Value of batch_tf elements: nan
Max Ratio Diff/Value batch_k: inf
Max Ratio Diff/Value batch_tf: inf




Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Mean Difference of elements: 0.6664692563237622
Avg Ratio Diff/Value of batch_k elements: nan
Avg Ratio Diff/Value of batch_tf elements: nan
Max Ratio Diff/Value batch_k: inf
Max Ratio Diff/Value batch_tf: inf




Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Mean Difference of elements: 0.15664452696219086
Avg Ratio Diff/Value of batch_k elemen

Avg Ratio Diff/Value of tp_eval elements: 0.1782871826517149
Avg Ratio Diff/Value of n_eval elements: 0.1315850072372695
Max Ratio Diff/Value tp_eval: 0.9671673789439489
Max Ratio Diff/Value n_eval: 0.49165484813151056



Diff Evals: [    5.20730362     5.56790147     3.40621145     4.56937643
     9.25544593    25.27344821    40.06320909   -88.08306225
   -19.60116135 -1226.24434856]
5/5 [==============================] - 0s 22ms/step - loss: 3.0399 - accuracy: 0.9000 - val_loss: 4.7454 - val_accuracy: 0.9000


/home/leo/Desktop/Thesis/New_Repo/rsc.py:227: RuntimeWarning: divide by zero encountered in true_divide
  diff_rsum_avg = diff_rsum / tf.size(t1, out_type=tf.dtypes.float64)
/home/leo/Desktop/Thesis/New_Repo/rsc.py:227: RuntimeWarning: invalid value encountered in true_divide
  diff_rsum_avg = diff_rsum / tf.size(t1, out_type=tf.dtypes.float64)
/home/leo/Desktop/Thesis/New_Repo/rsc.py:228: RuntimeWarning: divide by zero encountered in true_divide
  print(f"Mean Difference of elements: {diff_rsum_avg}")
/home/leo/Desktop/Thesis/New_Repo/rsc.py:228: RuntimeWarning: invalid value encountered in true_divide
  print(f"Mean Difference of elements: {diff_rsum_avg}")


In [62]:
in_shape = model.layers[0].input_shape[-2:]
print(in_shape)
np.ones(shape=in_shape).shape

(28, 28)


(28, 28)

In [81]:
#train[0].shape
#train[1]
#clone.layers[1].sat_layer.get_config()
clone.layers[1].sat_layer._batch_input_shape

AttributeError: 'sat_layer' object has no attribute '_batch_input_shape'

In [45]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

In [44]:
tf.distribute.get_strategy()

In [46]:
#(None, 21, 5)[1:]*2
#np.zeros(shape=(None))
importlib.reload(rsc)
train_b, test_b = rsc.get_titanic_dataset(batch_size=3)
print(train_b)
print(model(np.array([[1,2,3,4] ])))

for ix, x in enumerate(train_b):
    print(x)

<BatchDataset shapes: ((None, None, 4), (None, None, 2)), types: (tf.float32, tf.int32)>
tf.Tensor([[0.69211704 0.3078829 ]], shape=(1, 2), dtype=float32)
(<tf.Tensor: shape=(3, 20, 4), dtype=float32, numpy=
array([[[ 61.    ,   0.    ,   0.    ,  32.3208],
        [ 38.    ,   0.    ,   1.    , 153.4625],
        [ 36.    ,   0.    ,   0.    ,  26.3875],
        [ 28.    ,   0.    ,   0.    ,  26.55  ],
        [ 22.    ,   0.    ,   0.    ,  10.5167],
        [ 28.    ,   3.    ,   1.    ,  25.4667],
        [ 28.    ,   0.    ,   0.    ,   7.8958],
        [ 30.    ,   0.    ,   0.    ,   7.8958],
        [ 34.    ,   1.    ,   0.    ,  21.    ],
        [  3.    ,   4.    ,   2.    ,  31.3875],
        [ 31.    ,   1.    ,   0.    ,  57.    ],
        [ 16.    ,   0.    ,   0.    ,   8.05  ],
        [ 28.    ,   3.    ,   1.    ,  25.4667],
        [ 39.    ,   0.    ,   0.    ,   0.    ],
        [ 28.    ,   0.    ,   0.    , 227.525 ],
        [ 19.    ,   0.    ,   0.    ,   7

InvalidArgumentError: Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [7,4], [batch]: [20,4]